# Ejercicios de agua subterránea

En este _notebook_ se guardan las funciones que se utilizan en los ejercicios de agua subterránea, de manera que se pueden importar directamente en cualquier otro _notebook_ sin necesidad de reescribirlas. 

Para cargar estas funciones en otro _notebook_, se utiliza una de las funciones llamadas mágicas en terminología _Python_:
```Python
%run funciones_AguaSubterranea.ipynb
```
Esta línea sólo funciona si `funciones_AguaSubterranea.ipynb` está en la misma carpeta que el _notebook_ en el que se quiere cargar. En caso contrario, hay que utilizar el paquete `os` para definir en qué carpeta se encuentra `funciones_AguaSubterranea.ipynb`.
```Python
import os
os.chdir('ruta donde se encuentra `funciones_AguaSubterranea.ipynb')
%run funciones_AguaSubterranea.ipynb
```

__Índice__<br>

__[Método de infiltración de Green-Ampt](#Método-de-infiltración-de-Green-Ampt)__<br>

__[Propiedades del suelo](#Propiedades-del-suelo)__<br>

__[Altura hidráulica y dirección del flujo subterráneo](#Altura-hidráulica-y-dirección-del-flujo-subterráneo)__<br>

In [2]:
import numpy as np

import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('dark_background')
#plt.style.use('seaborn-whitegrid')

### Método de infiltración de Green-Ampt

In [10]:
def GreenAmpt(i, tc, ho, phi, theta_o, Ks, psi_ae, b=5.3, epsilon=0.001):
    """Se calcula la infiltración en un suelo para una precipitación constante mediante el método de Green-Ampt.
    
    Entradas:
    ---------
    i:       float. Intensidad de precipitación (cm/h)
    tc:      float. Tiempo de cálculo (h)
    ho:      float. Altura de la lámina de agua del encharcamiento en el inicio (cm)
    phi:     float. Porosidad (-)
    theta_o: float. Humedad del suelo en el inicio (-)
    Ks:      float. Conductividad saturada (cm/h)
    psi_ae:  float. Tensión del suelo para el punto de entrada de aire (cm)
    b:       float. Coeficiente para el cálculo de la tensión en el frente húmedo (cm)
    epsilo:  float. Error tolerable en el cálculo (cm)
    
    Salidas:
    --------
    Fc:      float. Infiltración acumulada en el tiempo de cálculo (cm)
    fc:      float. Tasa de infiltración en el tiempo de cálculo (cm/h)
    L:       float. Profundidad del frente húmedo en el tiempo de cálculo (cm)"""
    
    # calcular variables auxiliares
    Atheta = phi - theta_o                     # incremento de la humedad del suelo
    psi_f = (2 * b + 3) / (2 * b + 6) * psi_ae # tensión en el frente húmedo
    
    if ho > 0: # encharcamiento inicial
        tp = 0
        to = 0
    elif ho == 0: # NO hay encharcamiento inicial
        # tiempo hasta el encharcamiento
        tp = psi_f * Atheta * Ks / (i * (i - Ks))
        # infiltración acumulada cuando ocurre el encharcamiento
        Fp = tp * i
        # tiempo de inicio de la curva de infiltración
        to = tp - (Fp - psi_f * Atheta * np.log(1 + Fp / (psi_f * Atheta))) / Ks
    
    # infiltración acumulada en el tiempo de cálculo
    if tc <= tp:
        Fc = i * tc
    elif tc > tp:
        Fo = Ks * (tc - to)
        Fi = Ks * (tc - to) + psi_f * Atheta * np.log(1 + Fo / (psi_f * Atheta))
        while (Fi - Fo) > epsilon:
            Fo = Fi
            Fi = Ks * (tc - to) + psi_f * Atheta * np.log(1 + Fo / (psi_f * Atheta))
        Fc = Fi
    
    # tasa de infiltración en el tiempo de cálculo
    fc = Ks * (1 + psi_f * Atheta / Fc)
    
    # profundidad del frente de húmedo
    L = Fc / Atheta
    
    return Fc, fc, L

### Propiedades del suelo

In [28]:
def prop_suelo(m, ms, rho_m, D, L):
    """Se calculan las propiedades del suelo
    
    Entradas:
    ---------
    m:       float. Masa de la muestra de suelo (g)
    ms:      float. Masa seca de la muestra (g)
    rho_m:   float. Densidad del mineral (g/cm³)
    D:       float. Diámetro del cilindro de la muestra (cm)
    L:       float. Longitud del cilindro de la muestra (cm)
    
    Salidas:
    --------
    rho_b:   float. Densidad bruta (g/cm³)
    phi:     float. Porosidad (-)
    theta:   float. Humedad del suelo (-)
    Se:      float. Grado de saturación (-)"""
    
    rho_w = 1    # g/cm³
    
    # Calcular volúmenes necesarios
    Vt = np.pi / 4 * D**2 * L # volumen total
    Vw = (m - ms) / rho_w   # volumen de agua
    
    # Calcular propiedades del suelo
    rho_b = ms / Vt         # densidad del suelo
    phi = 1 - rho_b / rho_m # porosidad
    theta = Vw / Vt         # contenido de humedad del suelo
    Se = theta / phi        # grado de saturación
    
    return rho_b, phi, theta, Se

### Altura hidráulica y dirección del flujo subterráneo

In [59]:
def direc_flujo(psi_m, prof, Lten):
    """Se calcula la altura hidráulica en dos puntos del suelo y en función de ella la dirección del flujo subterráneo.
    
    Entradas:
    ---------
    psi_m:   list (2,). Valores medidos de la tensión (negativos) en los dos puntos del suelo (cm).
    prof:    list (2,). Profundidad (positivos) a la que se ubican los dos tensiómetros (cm)
    Lten:    float. Longitud del tensiómetro (cm)
    
    Salida:
    -------
    H:       list (2,). Altura de energía en los dos puntos (cm)"""
    
    # tensión en el bulbo de los tensiómetros
    psi = psi_m + Lten
    
    # altura potencial relativa a la superficie
    z = - prof
    
    # altura de energía
    H = z + psi
    print('Ha = {0} cm\tHb = {1} cm'.format(H[0], H[1]))
    
    # definir dirección del flujo
    if H[0] > H[1]:
        print('flujo en dirección a B')
    elif H[0] == H[1]:
        print('no hay flujo')
    elif H[0] < H[1]:
        print('flujo en dirección a A')
        
    return H